# Phase 2: Implementing a database for VictoryRide Transport Services

In this section, we will dive into the implementation phase of the VictoryRide Transport Services database. Here, we will focus on the crucial aspects of database creation and data management using Data Definition Statements (DDS) and Data Manipulation Statements (DMS). We will explore code snippets with brief explanation that demonstrate how to define the structure of the database, create tables, enforce constraints, and insert data. Additionally, we will delve into the realm of querying the database using DMS to extract meaningful insights, validate data integrity, and ensure the effectiveness of the implemented constraints. Through these code snippets, we will gain hands-on experience in implementing a robust and well-structured database solution for VictoryRide Transport Services.

    1. CREATE database and tables

In [ ]:
CREATE DATABASE IF NOT EXISTS VictoryRide;

USE VictoryRide
CREATE TABLE IF NOT EXISTS LANGUAGE (
    LanguageCode CHAR(2) PRIMARY KEY,
    LanguageName VARCHAR(50)
    ) ENGINE = InnoDB;

    2. Setting PRIMARY & FOREIGN KEY constraints

        2.1. Tables with One Primary Key

In [ ]:
CREATE TABLE IF NOT EXISTS LOCATION_TYPE (
    LocTypeID CHAR(2) PRIMARY KEY,
    LocTypeDescription VARCHAR(40)
    ) ENGINE = InnoDB;

        2.2. Tables with Composite Primary Key

In [ ]:
CREATE TABLE IF NOT EXISTS COUNTRY_LANGUAGE (
    CountryCode CHAR(2),
    LanguageCode CHAR(2),
    PRIMARY KEY (CountryCode, LanguageCode)
    ) ENGINE = InnoDB;

        2.3. Foreign Key

In [ ]:
CREATE TABLE IF NOT EXISTS CARGO_VEHICLE (
    VIN CHAR(17) PRIMARY KEY,
    Carg_cfeet_capacity SMALLINT,
    Carg_tons_capacity DECIMAL(5,2),
    FOREIGN KEY (VIN) REFERENCES VEHICLE(VIN)
    ) ENGINE = InnoDB;

    3. Other constraints

        3.1. UNIQUE - It ensures that a specific column or set of columns contains unique values.

In [ ]:
CREATE TABLE IF NOT EXISTS COUNTRY (
    CountryCode CHAR(2) PRIMARY KEY,
    CountryName VARCHAR(50) UNIQUE
    ) ENGINE = InnoDB;

        3.2. NOT NULL - It ensures that a column cannot have a null value.

In [ ]:
CREATE TABLE IF NOT EXISTS OFFICIAL_ROLE (
    RoleID CHAR(2) PRIMARY KEY,
    RoleDescription VARCHAR(40) NOT NULL
    ) ENGINE = InnoDB;

        3.3. CHECK - It validates that a column's value meets certain conditions.

In [ ]:
CREATE TABLE IF NOT EXISTS PASSENGER_VEHICLE (
    VIN CHAR(17) PRIMARY KEY,
    Pass_seat_capacity TINYINT,
    CHECK (Pass_seat_capacity < 6)
    ) ENGINE = InnoDB;

        3.4. AUTO INCREMENT - It’s used to automatically generate unique values for a column, typically used for primary keys.

In [ ]:
CREATE TABLE IF NOT EXISTS LOCATION (
    LocationID INT PRIMARY KEY AUTO_INCREMENT,
		Postcode CHAR(4)
    ) AUTO_INCREMENT = 1
ENGINE = InnoDB;

    4. Populate and Modify data

        4.1. INSERT - used to insert new records into a table

In [ ]:
USE VictoryRide;

INSERT INTO official_role VALUES
("CP", "Captain");

INSERT INTO maintenance_repair VALUES 
('AZ34','2000','M',STR_TO_DATE('18-06-2022', '%d-%m-%Y'),'2000.00','Oil Change');

        4.2. UPDATE - used to modify existing records in a table

In [ ]:
USE VictoryRide;

UPDATE official_language
SET LanguageCode = "en"
WHERE OfficialID = "SPN12345";

    5. Test and Validate
Test the database by executing queries, validating constraints, and performing various operations to ensure everything works as intended.

Example 1:  Find the Locations whose city names consist of one or more words, and the last word is four character long ending with a ‘k’. For each such location, display its street number, street name, and city.

In [ ]:
SELECT StreetNo, StreetName, City
FROM location
WHERE City LIKE '% ___k';

Example 2: Find the Date on which the most recent Trip(s) was(were) completed. Show the date as well as the vehicle model(s) and type(s) (description) involved in the trip(s).

In [ ]:
SELECT 
	DATE_FORMAT(T.EndTimeActual,'%d/%m/%Y') AS "Date",
	V.Veh_Model AS "Vehicle Model",
	VT.Veh_Type_Description AS "Vehicle Type"
FROM 
	trip AS T,
	vehicle AS V,
	vehicle_type AS VT
WHERE T.VIN = V.VIN 
	AND V.Veh_TypeID = VT.Veh_TypeID
ORDER BY T.EndTimeActual DESC
LIMIT 0,1;

Example 3: List the Vehicles which have been driven more than 1000 KM in total for VictoryRide trips. Display the vehicle registration numbers and total kilometres travelled. Show the list sorted by total kilometre travelled.

In [ ]:
SELECT V.Veh_RegoNum AS `Vehicle Registration Number`,
       SUM(T.EndOdometerKM - T.StartOdometerKM) AS `Total Kilometers Traveled`
FROM TRIP T, VEHICLE V
WHERE T.VIN = V.VIN
  AND T.EndOdometerKM - T.StartOdometerKM > 1000
GROUP BY T.VIN, V.Veh_RegoNum
ORDER BY `Total Kilometers Traveled` DESC;

Example 4: Find the Passenger Vehicles whose one or more (single) repair costs were more than the average repair cost of all vehicles. For each vehicle, display the vehicle registration number, make, model, seat capacity, and total repair cost. Note that if a vehicle went for multiple repairs, its total repair cost is the sum of all these (single) repair costs.

In [ ]:
SELECT
	V.Veh_RegoNum AS "Vehicle Registration Num",
	V.Veh_Make AS "Vehicle Make",
	V.Veh_Model AS "Vehicle Model",
	P.Pass_seat_capacity AS "Seat Capacity",
	CONCAT("$ ",SUM(MR.MR_Cost)) AS "Maitenance and Repair Cost"
FROM vehicle AS V,
	passenger_vehicle AS P,
    maintenance_repair AS MR
WHERE V.VIN = MR.VIN 
	AND V.VIN = P.VIN 
	AND V.Veh_For_Passenger = "Y"
GROUP BY V.VIN
HAVING
    SUM(MR.MR_Cost) > (SELECT AVG(MR_Cost) FROM maintenance_repair);

Example 5: A list of all Drivers who have not been involved in any trip yet (i.e., haven’t actually driven for any trips yet). Display the drivers’ full names, security clearance levels, and the languages they speak with at their highest proficiency levels. Note that a driver may speak in more than one language at his/her highest level of proficiency, which can be 5 or below.

In [ ]:
SELECT 
    CONCAT(D.DriverFirstName, ' ', D.DriverLastName) AS "Drivers Fullname",
    D.ClearanceLevel AS "Security Clearance Levels",
    L.LanguageName
FROM 
    driver D, language L, driver_language DL
WHERE 
    D.DriverLicenceNum = DL.DriverLicenceNum
    AND DL.LanguageCode = L.LanguageCode
    AND (DL.DriverLicenceNum, DL.DriverLanguageProf) IN (
        SELECT DriverLicenceNum, MAX(DriverLanguageProf)
        FROM driver_language
        GROUP BY DriverLicenceNum
    )
    AND NOT EXISTS (
        SELECT 1
        FROM trip T
        WHERE D.DriverLicenceNum = T.DriverLicenceNum
    );

    6. Optimize and Fine-tune
During the testing phase, you may identify areas where the database performance can be improved.
    
    7. Deploy, Monitor and Maitain
Once the database implementation is validated and optimized, it can be deployed to the production environment. Ongoing monitoring and maintenance of the database are essential to ensure its continued optimal performance and data integrity. Regular backups, security measures, and performance monitoring should be established to support the ongoing operation of the database.